<a href="https://colab.research.google.com/github/graylan0/purple-llama/blob/main/Purple_Llama_MemoryConstraint_Twitchio_llama2_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install fastapi
!pip install uvicorn
!pip install pysqlite3
!pip install twitchio
!pip install ratelimit
!pip install twitchio fastapi pydantic llama_cpp cachetools ratelimit
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.2-cp310-cp310-linux_x86_64.whl size=151112 sha256=f94f6d0c64d148bbe191ba0de0b1d2983a3ae9c4d1fe449ab2b7a1e91a391a76
  Stored in directory: /root/.cache/pip/wheels/91/91/77/1912a316aad35d0edcb42ee1aa9b7bc07d7190a5798d378805
Successfully built pysqlite3
ERROR: Could not find a version that satisfies the requirement llama_cpp (from versions: none)
ERROR: No matching distribution found for llama_cpp


In [9]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

--2023-09-29 05:33:56--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1696224836&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NjIyNDgzNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1Mm

In [3]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822227 sha256=2b142504c33b3aca38db2278613d33ceb61312e601cd0ebfcc9e8d53c6aabdbb
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [ ]:
import json
import asyncio
import logging
from concurrent.futures import ThreadPoolExecutor
from twitchio.ext import commands
from llama_cpp import Llama  # Assuming you have this package installed
import textwrap  # Add this line to import the textwrap module

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Initialize twitch_token and initial_channels with default values
twitch_token = None
initial_channels = ["freedomdao"]

# Load configuration from config.json
try:
    with open("config.json", "r") as f:
        config = json.load(f)
    twitch_token = config.get("TWITCH_TOKEN")
    initial_channels = config.get("INITIAL_CHANNELS", ["freedomdao"])
except Exception as e:
    logging.error(f"Failed to load config.json: {e}")

# Initialize Llama model
llm = Llama(
  model_path="llama-2-7b-chat.ggmlv3.q8_0.bin",
  n_gpu_layers=-1,
  n_ctx=3900,
)

executor = ThreadPoolExecutor(max_workers=3)

async def run_llm(prompt):
    return await asyncio.get_event_loop().run_in_executor(executor, lambda: llm(prompt, max_tokens=900)['choices'][0]['text'])

class TwitchBot(commands.Bot):
    def __init__(self):
        super().__init__(token=twitch_token, prefix="!", initial_channels=initial_channels)

    async def event_ready(self):
        print(f'Logged in as | {self.nick}')

    async def event_message(self, message):
        await self.handle_commands(message)

    @commands.command(name="llama")
    async def llama_command(self, ctx):
        prompt = ctx.message.content.replace("!llama ", "")
        reply = await run_llm(prompt)

        # Split the reply into chunks of 500 characters
        chunks = textwrap.wrap(reply, 500)

        # Send each chunk as a separate message
        for chunk in chunks:
            await ctx.send(chunk)

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()

    bot = TwitchBot()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(bot.run())


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Logged in as | freedomdao


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/twitchio/client.py", line 208, in wrapped
    await func(*args)
  File "<ipython-input-1-e94290ada3d8>", line 45, in event_message
    await self.handle_commands(message)
  File "/usr/local/lib/python3.10/dist-packages/twitchio/ext/commands/bot.py", line 339, in handle_commands
    context = await self.get_context(message)
  File "/usr/local/lib/python3.10/dist-packages/twitchio/ext/commands/bot.py", line 293, in get_context
    return cls(message=message, prefix=prefix, valid=False, bot=self)
  File "/usr/local/lib/python3.10/dist-packages/twitchio/ext/commands/core.py", line 537, in __init__
    self._ws = self.author._ws
AttributeError: 'NoneType' object has no attribute '_ws'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/twitchio/client.py", line 208, in wrapped
    await func(*args)
  File "<ipython-input-1-e94290ada3d8>", line 45, in event_message
    await self